In [ ]:
"""
RUL_ML_Framework
File Name: cae_sd.py

Description: convolutional autoencoder reducing spatial dimension for statistical features.

Author: junghwan lee

Email: jhrrlee@gmail.com
Date Created: 2023.10.16

"""

In [1]:
from keras.layers import Input, Conv1D, BatchNormalization, UpSampling1D, Flatten, Dense, Reshape
from keras.models import Model
from tensorflow.keras.initializers import HeNormal

2024-09-11 21:04:13.249975: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 21:04:13.563819: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 21:04:14.345218: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
def create_cae_sd(no_of_features, fixed_channel_dim=1, use_bn=True, kernel_size=3, no_of_filters=128):
    input_shape = (no_of_features, fixed_channel_dim)
    initializer = HeNormal(seed=42)

    # Encoder
    inputs = Input(shape=input_shape)

    x = Conv1D(no_of_filters, kernel_size, activation='relu', padding='same', strides=2, kernel_initializer=initializer)(inputs)
    if use_bn:
        x = BatchNormalization()(x)
    x = Conv1D(no_of_filters//2, kernel_size, activation='relu', padding='same', strides=2, kernel_initializer=initializer)(x)
    if use_bn:
        x = BatchNormalization()(x)
    x = Conv1D(no_of_filters//4, kernel_size, activation='relu', padding='same', strides=2, kernel_initializer=initializer)(x)
    if use_bn:
        x = BatchNormalization()(x)
    x = Conv1D(no_of_filters//8, kernel_size, activation='relu', padding='same', strides=1, kernel_initializer=initializer)(x)
    if use_bn:
        x = BatchNormalization()(x)

    encoded = Conv1D(1, 1, activation='linear', padding='same', kernel_initializer=initializer)(x)

    # Decoder
    x = UpSampling1D(2)(encoded)  # double the feature dimension
    x = Conv1D(no_of_filters//8, kernel_size, activation='relu', padding='same', kernel_initializer=initializer)(x)
    if use_bn:
        x = BatchNormalization()(x)
    x = UpSampling1D(2)(x)  # double the feature dimension
    x = Conv1D(no_of_filters//4, kernel_size, activation='relu', padding='same', kernel_initializer=initializer)(x)
    if use_bn:
        x = BatchNormalization()(x)
    x = UpSampling1D(2)(x)  # double the feature dimension
    x = Conv1D(no_of_filters//2, kernel_size, activation='relu', padding='same', kernel_initializer=initializer)(x)
    if use_bn:
        x = BatchNormalization()(x)
    x = Conv1D(no_of_filters, kernel_size, activation='relu', padding='same', kernel_initializer=initializer)(x)
    x = Flatten()(x)
    decoded = Dense(no_of_features * fixed_channel_dim, activation='linear')(x)
    decoded = Reshape((no_of_features, fixed_channel_dim))(decoded)

    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)

    return autoencoder, encoder


: 